<a href="https://colab.research.google.com/github/fongbubble/EFIMM0142_NBA_CEC_DEA/blob/main/CEC_NBA_wo_schedule.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

$$\text{Supanu Tanatammatid 2546980}$$

## Package

In [1]:
!pip install pulp
import pulp
!apt-get install -y -qq glpk-utils

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 73.1 MB/s eta 0:00:00
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 123630 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_5.0-1_amd64.deb ...
Unpacking 

In [2]:
from pulp import GLPK
import pandas as pd # Library to handle data
import numpy as np
import math
from statistics import mean

In [3]:
url = "https://raw.githubusercontent.com/fongbubble/EFIMM0142_NBA_CEC_DEA/main/nba_updated_withrest1.csv"

# Read the CSV file directly from the GitHub link
data = pd.read_csv(url)
data = data.rename(columns={
    "Head Coach's Experience": "Head_Coachs_Experience",
    "Head Coach's Ability": "Head_Coachs_Ability",
    "No. of Games with Rest ": "No_of_Games_with_Rest"  # Rename this column as well
})

## DEA wo schedule


In [4]:
# Initialize a weights matrix to store results (16 rows for each team, 10 columns for different metrics)
weights_matrix = np.empty((16, 9))

# Initialize an array to store objective values for 16 teams
objective_values = np.empty(16)  # Array of size (16,)

# Solve optimization problems and fill in weights_matrix
for k in range(9):
    for i in range(16):
        model = pulp.LpProblem("NBA", pulp.LpMaximize)  # Create an LP maximization problem

        # Create decision variables
        u1 = pulp.LpVariable("u1", lowBound=0, upBound=None, cat='Continuous')  # Games Won
        u2 = pulp.LpVariable("u2", lowBound=0, upBound=None, cat='Continuous')  # Playoff -Depth
        u3 = pulp.LpVariable("u3", lowBound=0, upBound=None, cat='Continuous')  # Average Margin of Victory
        v1 = pulp.LpVariable("v1", lowBound=0, upBound=None, cat='Continuous')  # AFTbudget
        v2 = pulp.LpVariable("v2", lowBound=0, upBound=None, cat='Continuous')  # ABTBudget
        v3 = pulp.LpVariable("v3", lowBound=0, upBound=None, cat='Continuous')  # Head Coach's Experience
        v4 = pulp.LpVariable("v4", lowBound=0, upBound=None, cat='Continuous')  # Head Coach's Ability
        v5 = pulp.LpVariable("v5", lowBound=0, upBound=None, cat='Continuous')  # Roster average age
        v6 = pulp.LpVariable("v6", lowBound=0, upBound=None, cat='Continuous')  # Average years of experience in the league

        # Objective function
        model += (
            u1 * data["Games Won"].iloc[i] +
            u2 * data["Playoff -Depth"].iloc[i] +
            u3 * data["Average Margin of Victory"].iloc[i]
        )

        # First constraint (equality)
        model += (
            v1 * data["AFTbudget"].iloc[i] +
            v2 * data["ABTBudget"].iloc[i] +
            v3 * data["Head_Coachs_Experience"].iloc[i] +
            v4 * data["Head_Coachs_Ability"].iloc[i] +
            v5 * data["Roster average age"].iloc[i] +
            v6 * data["Average years of experience in the league"].iloc[i] == 1
        )

        # Additional constraints
        for j in range(16):
            model += (
                u1 * data["Games Won"].iloc[j] +
                u2 * data["Playoff -Depth"].iloc[j] +
                u3 * data["Average Margin of Victory"].iloc[j] -
                v1 * data["AFTbudget"].iloc[j] -
                v2 * data["ABTBudget"].iloc[j] -
                v3 * data["Head_Coachs_Experience"].iloc[j] -
                v4 * data["Head_Coachs_Ability"].iloc[j] -
                v5 * data["Roster average age"].iloc[j] -
                v6 * data["Average years of experience in the league"].iloc[j] <= 0
            )

        # Solve the problem
        status = model.solve(pulp.GLPK(msg=True, options=['--ranges', 'sensitivity_wo_schedule.txt']))

        # Displaying Optimal Decision Variables & Reduced Cost per Variable
        list_weights = []
        for v in model.variables():
            list_weights.append(v.varValue)  # Collect optimal values of the decision variables

        # Storing decision variables in weights_matrix
        # Explicitly assign weights to specific columns to maintain proper mapping
        weights_matrix[i, 0] = list_weights[0]  # m1 -> Column 0 (Scenario_1)
        weights_matrix[i, 1] = list_weights[1]  # m2 -> Column 1 (Scenario_2)
        weights_matrix[i, 2] = list_weights[2]  # m3 -> Column 2 (Scenario_3)
        weights_matrix[i, 3] = list_weights[3]  # n1 -> Column 3 (Scenario_4)
        weights_matrix[i, 4] = list_weights[4]  # n2 -> Column 4 (Scenario_5)
        weights_matrix[i, 5] = list_weights[5]  # n3 -> Column 5 (Scenario_6)
        weights_matrix[i, 6] = list_weights[6]  # n4 -> Column 6 (Scenario_7)
        weights_matrix[i, 7] = list_weights[7]  # n5 -> Column 7 (Scenario_8)
        weights_matrix[i, 8] = list_weights[8]  # n6 -> Column 8 (Scenario_9)

        # Collect the objective value for each team
        objective_value = pulp.value(model.objective)
        objective_values[i] = objective_value  # Store the value in the array

team_names = data['Team'].tolist()  # Assuming 'data' contains a column called 'Team' with the team names

# Create generic column names for each scenario
scenario_names = [f'Scenario_{k+1}' for k in range(9)]

# Create DataFrame with team names as row labels and generic column names for scenarios
weights_df = pd.DataFrame(weights_matrix, index=team_names, columns=scenario_names)

# Reshape objective_values to be 16x1 array
objective_values_array = objective_values.reshape(16, 1)

# Output the objective values as a 16x1 array
print("Objective Values Array (16x1):")
print(objective_values_array)


Objective Values Array (16x1):
[[1.00000028]
 [1.0000023 ]
 [0.999999  ]
 [1.00000177]
 [0.8747571 ]
 [1.        ]
 [0.851205  ]
 [0.8053689 ]
 [0.9723903 ]
 [0.8153088 ]
 [0.9450454 ]
 [0.999999  ]
 [0.8311762 ]
 [1.0000002 ]
 [0.8180209 ]
 [0.74014   ]]


In [5]:
# Calculate New Weights using Second Objective Model
CEC_weights_matrix = np.empty((16, 9))

# Solve optimization problems and fill in CEC_weights_matrix
for k in range(9):
    for i in range(16):
        Second_Objective_model = pulp.LpProblem("NBA_CEC", pulp.LpMaximize)

        # Define decision variables
        u1 = pulp.LpVariable("u1", lowBound=0, upBound=None, cat='Continuous')  # Games Won
        u2 = pulp.LpVariable("u2", lowBound=0, upBound=None, cat='Continuous')  # Playoff -Depth
        u3 = pulp.LpVariable("u3", lowBound=0, upBound=None, cat='Continuous')  # Average Margin of Victory
        v1 = pulp.LpVariable("v1", lowBound=0, upBound=None, cat='Continuous')  # AFTbudget
        v2 = pulp.LpVariable("v2", lowBound=0, upBound=None, cat='Continuous')  # ABTBudget
        v3 = pulp.LpVariable("v3", lowBound=0, upBound=None, cat='Continuous')  # Head Coach's Experience
        v4 = pulp.LpVariable("v4", lowBound=0, upBound=None, cat='Continuous')  # Head Coach's Ability
        v5 = pulp.LpVariable("v5", lowBound=0, upBound=None, cat='Continuous')  # Roster average age
        v6 = pulp.LpVariable("v6", lowBound=0, upBound=None, cat='Continuous')  # Average years of experience in the league

        # Define the objective function
        # Define the objective function
        Second_Objective_model += (
            u1 * data["Games Won"].sum() +
            u2 * data["Playoff -Depth"].sum() +
            u3 * data["Average Margin of Victory"].sum()
            - (v1 * data["AFTbudget"].sum() +
            v2 * data["ABTBudget"].sum() +
            v3 * data["Head_Coachs_Experience"].sum() +
            v4 * data["Head_Coachs_Ability"].sum() +
            v5 * data["Roster average age"].sum() +
            v6 * data["Average years of experience in the league"].sum())
        )

        # Define the equality constraint
        Second_Objective_model += (
            v1 * data["AFTbudget"].iloc[i] +
            v2 * data["ABTBudget"].iloc[i] +
            v3 * data["Head_Coachs_Experience"].iloc[i] +
            v4 * data["Head_Coachs_Ability"].iloc[i] +
            v5 * data["Roster average age"].iloc[i] +
            v6 * data["Average years of experience in the league"].iloc[i] == 1
        )

        # Define the inequalities for each team
        for j in range(16):
            Second_Objective_model += (
                u1 * data["Games Won"].iloc[j] +
                u2 * data["Playoff -Depth"].iloc[j] +
                u3 * data["Average Margin of Victory"].iloc[j]
                - (v1 * data["AFTbudget"].iloc[j] +
                   v2 * data["ABTBudget"].iloc[j] +
                   v3 * data["Head_Coachs_Experience"].iloc[j] +
                   v4 * data["Head_Coachs_Ability"].iloc[j] +
                   v5 * data["Roster average age"].iloc[j] +
                   v6 * data["Average years of experience in the league"].iloc[j]) <= 0
            )

        # Additional constraint using the list of objective values
        Second_Objective_model += (
            u1 * data["Games Won"].iloc[i] +
            u2 * data["Playoff -Depth"].iloc[i] +
            u3 * data["Average Margin of Victory"].iloc[i]
            - (objective_values_array[i, 0] * (
                v1 * data["AFTbudget"].iloc[i] +
                v2 * data["ABTBudget"].iloc[i] +
                v3 * data["Head_Coachs_Experience"].iloc[i] +
                v4 * data["Head_Coachs_Ability"].iloc[i] +
                v5 * data["Roster average age"].iloc[i] +
                v6 * data["Average years of experience in the league"].iloc[i])) == 0
        )

        # Solve the model
        status = Second_Objective_model.solve()
        print("Model Status:{}".format(pulp.LpStatus[Second_Objective_model.status]))

        # Save the optimal variable values in the weight matrix
        new_list_weights = []
        for v in Second_Objective_model.variables():
            # Ensure non-negative values for decision variables
            v.varValue = max(0, v.varValue)
            new_list_weights.append(v.varValue)
            print(v.name, "=", v.varValue)

        # Store the weights into the CEC_weights_matrix
        CEC_weights_matrix[i, 0] = new_list_weights[0]  # u1 -> Column 0 (Scenario_1)
        CEC_weights_matrix[i, 1] = new_list_weights[1]  # u2 -> Column 1 (Scenario_2)
        CEC_weights_matrix[i, 2] = new_list_weights[2]  # u3 -> Column 2 (Scenario_3)
        CEC_weights_matrix[i, 3] = new_list_weights[3]  # v1 -> Column 3 (Scenario_4)
        CEC_weights_matrix[i, 4] = new_list_weights[4]  # v2 -> Column 4 (Scenario_5)
        CEC_weights_matrix[i, 5] = new_list_weights[5]  # v3 -> Column 5 (Scenario_6)
        CEC_weights_matrix[i, 6] = new_list_weights[6]  # v4 -> Column 6 (Scenario_7)
        CEC_weights_matrix[i, 7] = new_list_weights[7]  # v5 -> Column 7 (Scenario_8)
        CEC_weights_matrix[i, 8] = new_list_weights[8]  # v6 -> Column 8 (Scenario_9)

        # Display constraints for debugging (optional)
        print("\nConstraint")
        for name, c in Second_Objective_model.constraints.items():
            print(name, ":", c)

# Create DataFrame with team names as row labels and generic column names for scenarios
CEC_weights_df = pd.DataFrame(CEC_weights_matrix, index=team_names, columns=[f'Scenario_{k+1}' for k in range(9)])

# Output the weights DataFrame
print("Final CEC Weights Matrix with Team Names and Scenario Labels:")
print(CEC_weights_df)


Model Status:Optimal
u1 = 0.013660761
u2 = 0.02514231
u3 = 0
v1 = 0
v2 = 0.038764851
v3 = 0
v4 = 0.9607155
v5 = 0.0073617467
v6 = 0

Constraint
_C1 : 31.3*v1 + 2.55*v2 + 5.1*v3 + 0.738*v4 + 26.1*v5 + 4.7*v6 = 1.0
_C2 : 64*u1 + 5*u2 + 11.3*u3 - 31.3*v1 - 2.55*v2 - 5.1*v3 - 0.738*v4 - 26.1*v5 - 4.7*v6 <= -0.0
_C3 : 57*u1 + 2*u2 + 5.3*u3 - 30.4*v1 - 2.61*v2 - 6.72*v3 - 0.556*v4 - 26.3*v5 - 4.88*v6 <= -0.0
_C4 : 57*u1 + 2*u2 + 7.4*u3 - 14.0*v1 - 5.64*v2 - 5.76*v3 - 0.446*v4 - 24.7*v5 - 3.41*v6 <= -0.0
_C5 : 56*u1 + 3*u2 + 6.5*u3 - 23.8*v1 - 3.51*v2 - 5.66*v3 - 0.537*v4 - 25.6*v5 - 5.1*v6 <= -0.0
_C6 : 51*u1 + u2 + 3.3*u3 - 31.2*v1 - 4.01*v2 - 6.27*v3 - 0.527*v4 - 28.4*v5 - 6.71*v6 <= -0.0
_C7 : 50*u1 + 4*u2 + 2.2*u3 - 23.3*v1 - 4.27*v2 - 6.43*v3 - 0.521*v4 - 25.6*v5 - 4.73*v6 <= -0.0
_C8 : 50*u1 + 2*u2 + 4.6*u3 - 22.0*v1 - 3.98*v2 - 6.82*v3 - 0.574*v4 - 26.4*v5 - 5.0*v6 <= -0.0
_C9 : 49*u1 + u2 + 2.6*u3 - 29.5*v1 - 3.46*v2 - 7.55*v3 - 0.569*v4 - 27.9*v5 - 7.05*v6 <= -0.0
_C10 : 49*u1 + u2 

In [6]:
# Initialize an efficiency table with shape (16, 16) to cover all rows (teams) and scenarios (weights)
CEC_efficiency_table = np.empty((16, 16))

# List to store row-wise averages for efficiency scores
CEC_average_list = []

# Iterate through rows (teams) and columns (scenarios) to calculate efficiency values
for team in range(16):
    row_values = []  # To store the values for each team across all scenarios for averaging
    for scenario in range(16):
        # Calculate efficiency using the available weights from weights_matrix and the dataset features from data
        # Use the scenario index to get the weights and apply them to the specific team's data

        # Calculate the numerator (outputs)
        numerator = (
            (CEC_weights_matrix[scenario, 0] * data['Games Won'].iloc[team]) +
            (CEC_weights_matrix[scenario, 1] * data['Playoff -Depth'].iloc[team]) +
            (CEC_weights_matrix[scenario, 2] * data['Average Margin of Victory'].iloc[team])
        )

        # Calculate the denominator (inputs)
        denominator = (
            (CEC_weights_matrix[scenario, 3] * data['AFTbudget'].iloc[team]) +
            (CEC_weights_matrix[scenario, 4] * data['ABTBudget'].iloc[team]) +
            (CEC_weights_matrix[scenario, 5] * data["Head_Coachs_Experience"].iloc[team]) +
            (CEC_weights_matrix[scenario, 6] * data["Head_Coachs_Ability"].iloc[team]) +
            (CEC_weights_matrix[scenario, 7] * data['Roster average age'].iloc[team]) +
            (CEC_weights_matrix[scenario, 8] * data['Average years of experience in the league'].iloc[team])
        )

        # Avoid division by zero by setting denominator to a small positive value if it's zero
        if denominator == 0:
            denominator = 1e-10  # A very small positive number

        # Calculate the efficiency value
        value = numerator / denominator

        # Assign the calculated value to the efficiency table
        CEC_efficiency_table[team, scenario] = value

        # Append value to row_values list for averaging later
        row_values.append(value)

    # Calculate the average for the row and append to average_list
    average = np.mean(row_values)
    CEC_average_list.append(average)


In [7]:
# Create the efficiency DataFrame
CEC_efficiency_df = pd.DataFrame(CEC_efficiency_table, index=team_names, columns=[name + "_weight" for name in team_names])

# Print the efficiency table
print("Efficiency Table with Team Names:")
print(CEC_efficiency_df.to_string(formatters={col: '{:.2f}'.format for col in CEC_efficiency_df.columns}))

# Export the efficiency DataFrame to a CSV file
csv_filename = "2nd_efficiency_table_wo_rest.csv"  # Define the filename
CEC_efficiency_df.to_csv(csv_filename, index=True)  # Export the DataFrame with index

print(f"\nThe efficiency table has been exported to '{csv_filename}'.")


Efficiency Table with Team Names:
                       Boston Celtics_weight Denver Nuggets_weight Oklahoma City Thunder_weight Minnesota Timberwolves_weight Los Angeles Clippers_weight Dallas Mavericks_weight New York Knicks_weight Milwaukee Bucks_weight New Orleans Pelicans_weight Phoenix Suns_weight Cleveland Cavaliers_weight Indiana Pacers_weight Los Angeles Lakers_weight Orlando Magic_weight Philadelphia 76ers_weight Miami Heat_weight
Boston Celtics                          1.00                  1.00                         1.00                          1.00                        1.00                    1.00                   1.00                   0.88                        1.00                0.88                       0.96                  1.00                      1.00                 0.68                      1.00              1.00
Denver Nuggets                          1.00                  1.00                         1.00                          1.00                 

In [8]:
# Create a list of tuples (team, efficiency)
CEC_team_efficiency = list(zip(data['Team'], CEC_average_list))

# Sort the list by efficiency in descending order
sorted_CEC_team_efficiency = sorted(CEC_team_efficiency, key=lambda x: x[1], reverse=True)

# Print the sorted cross-efficiency scores
for team, efficiency in sorted_CEC_team_efficiency:
    print(f"Cross-efficiency score for {team} = {round(efficiency, 3)}")

Cross-efficiency score for Oklahoma City Thunder = 1.0
Cross-efficiency score for Boston Celtics = 0.963
Cross-efficiency score for Minnesota Timberwolves = 0.952
Cross-efficiency score for Denver Nuggets = 0.939
Cross-efficiency score for Cleveland Cavaliers = 0.869
Cross-efficiency score for Indiana Pacers = 0.858
Cross-efficiency score for New Orleans Pelicans = 0.85
Cross-efficiency score for Dallas Mavericks = 0.834
Cross-efficiency score for New York Knicks = 0.797
Cross-efficiency score for Los Angeles Clippers = 0.771
Cross-efficiency score for Orlando Magic = 0.759
Cross-efficiency score for Phoenix Suns = 0.741
Cross-efficiency score for Milwaukee Bucks = 0.737
Cross-efficiency score for Los Angeles Lakers = 0.735
Cross-efficiency score for Philadelphia 76ers = 0.733
Cross-efficiency score for Miami Heat = 0.689


In [9]:
# Use np.isclose to check for values that are close to 1.0
tolerance = 1e-6
close_to_one = np.isclose(CEC_efficiency_table, 1.0, atol=tolerance)

# Count the number of values close to 1.0 in each row
count_close_to_one_per_row = np.sum(close_to_one, axis=1)

# Step 1: Find the row with the maximum count of values close to 1.0
row_with_max_ones = np.argmax(count_close_to_one_per_row)

# Step 2: Rescale Matrix
# Create a new array and divide each column by the corresponding value in the row with the maximum number of ones
Rescale_Matrix = np.zeros_like(CEC_efficiency_table)
for i in range(CEC_efficiency_table.shape[1]):
    # Divide by the element in the identified row and current column
    Rescale_Matrix[:, i] = CEC_efficiency_table[:, i] / (CEC_efficiency_table[row_with_max_ones, i] + 1e-10)

# Round the rescaled matrix to 4 decimal places
Rescale_Matrix = np.round(Rescale_Matrix, 4)

# Step 3: Compute Geometric Mean for Each DMU (Rows)
geometric_mean = [np.prod(Rescale_Matrix[i, :]) ** (1 / Rescale_Matrix.shape[1]) for i in range(Rescale_Matrix.shape[0])]

# Step 4: Calculate Root Sum of Squared Deviations
# Initialize list to store the root sum of squared deviations
root_sum_squared_list = []

# Iterate over each column
for col in range(Rescale_Matrix.shape[1]):
    column = Rescale_Matrix[:, col]  # Select the current column
    mean = geometric_mean[col]  # Access corresponding geometric mean for each DMU (using correct index)

    # Calculate square root of the sum of squared deviations
    deviations = math.sqrt(sum((x - mean) ** 2 for x in column))

    # Append the root of sum of squared deviations for the column
    root_sum_squared_list.append(deviations)

# Step 5: Calculate Evaluation Consensus Degree (ECD)
ECD = [1 / (1 + x) for x in root_sum_squared_list]

# Step 6: Calculate Consensus Efficiency Cross (CEC) Results
# Calculate the sum of all ECD values (total power)
total_power = sum(ECD)

# Calculate the product of each row, raised to the power of the total power
CEC_Results = []
for i in range(len(ECD)):
    row = Rescale_Matrix[i]
    # Use np.power to raise each element in the row to the ECD[i] power
    product = np.prod(np.power(row + 1e-10, ECD[i]))  # Adding epsilon to ensure numerical stability
    result = np.power(product, 1 / total_power)
    CEC_Results.append(result)

# Convert CEC_Results to a NumPy array and round to 4 decimal places
CEC_Results = np.array(CEC_Results).round(decimals=4)

# Step 7: Rank All DMUs Based on Final Aggregated Efficiency Scores
# Create a DataFrame to display the results
df_results = pd.DataFrame({
    'Team': team_names,
    'Consensus Efficiency Score': CEC_Results
})

# Sort the teams by their consensus efficiency score in descending order to rank them
df_sorted = df_results.sort_values(by='Consensus Efficiency Score', ascending=False).reset_index(drop=True)

# Step 8: Cap CEC_Results values greater than 1.0 to 1.0000 for final output
# Update the consensus efficiency scores in the sorted DataFrame
#df_sorted['Consensus Efficiency Score'] = df_sorted['Consensus Efficiency Score'].clip(upper=1.0000)

# Display the rankings
print(df_sorted)

                      Team  Consensus Efficiency Score
0    Oklahoma City Thunder                      1.0000
1           Boston Celtics                      0.9590
2   Minnesota Timberwolves                      0.9469
3           Denver Nuggets                      0.9275
4     New Orleans Pelicans                      0.8639
5      Cleveland Cavaliers                      0.8512
6           Indiana Pacers                      0.8400
7         Dallas Mavericks                      0.8252
8          New York Knicks                      0.7830
9            Orlando Magic                      0.7747
10    Los Angeles Clippers                      0.7594
11            Phoenix Suns                      0.7344
12      Philadelphia 76ers                      0.7325
13         Milwaukee Bucks                      0.7315
14      Los Angeles Lakers                      0.7251
15              Miami Heat                      0.7154


In [10]:
# Convert the ECD array to a pandas DataFrame
ECD_df = pd.DataFrame(ECD)

# Add a column for team names
ECD_df.insert(0, 'Team', team_names)

# Export the DataFrame to a CSV file
ECD_df.to_csv('ECD_output_wo_rest.csv', index=False)

In [11]:
# Convert CEC_average_list to a DataFrame
CEC_average_df = pd.DataFrame(CEC_average_list, columns=['Average Cross-efficiency'], index=team_names)


# Merge the Consensus Efficiency Score DataFrame and Cross-Efficiency Score DataFrame
merged_df = pd.merge(df_sorted, CEC_average_df, left_on='Team', right_index=True)

# Display the combined DataFrame
print(merged_df)

                      Team  Consensus Efficiency Score  \
0    Oklahoma City Thunder                      1.0000   
1           Boston Celtics                      0.9590   
2   Minnesota Timberwolves                      0.9469   
3           Denver Nuggets                      0.9275   
4     New Orleans Pelicans                      0.8639   
5      Cleveland Cavaliers                      0.8512   
6           Indiana Pacers                      0.8400   
7         Dallas Mavericks                      0.8252   
8          New York Knicks                      0.7830   
9            Orlando Magic                      0.7747   
10    Los Angeles Clippers                      0.7594   
11            Phoenix Suns                      0.7344   
12      Philadelphia 76ers                      0.7325   
13         Milwaukee Bucks                      0.7315   
14      Los Angeles Lakers                      0.7251   
15              Miami Heat                      0.7154   

    Average C